## Mistral with TransformerLens

In [1]:
import torch as t

In [3]:
t.cuda.is_available()

True

In [4]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
import matplotlib.pyplot as plt
import torch
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

/home/t-sgolechha/miniconda3/envs/mistral/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import sys
sys.path.append('../local_tf_lens/TransformerLens/')

In [14]:
from transformer_lens import HookedTransformer

In [15]:
torch.cuda.is_available()

True

In [16]:
tl_mistral = HookedTransformer.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:58<00:00, 29.22s/it]


Loaded pretrained model mistralai/Mistral-7B-v0.1 into HookedTransformer


In [17]:
tl_mistral.cfg.device

device(type='cuda')

In [18]:
tl_mistral.generate('What did the dog say to the telephone?', max_new_tokens=30)

100%|██████████| 30/30 [00:02<00:00, 12.21it/s]


'What did the dog say to the telephone?\n\nRin-Gin-Rin-Gin-Rin\n\nFur is not a furry word.\n\nif'

In [19]:
_, cache = tl_mistral.run_with_cache(
    'What is the answer to life, the universe and everything? 42',
)

In [21]:
cache['blocks.1.hook_resid_post'].shape

torch.Size([1, 16, 4096])

In [23]:
def dummy_hook(activation, hook):
    print(activation.shape)
    return

In [24]:
_ = tl_mistral.run_with_hooks(
    'Hello',
    fwd_hooks=[(lambda name: 'blocks.1.hook_resid_post' in name, dummy_hook)]
)

torch.Size([1, 2, 4096])
